In [1]:
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from itertools import cycle
import numpy as np
from sklearn.metrics import (
    roc_curve,
    precision_recall_curve,
    auc,
    average_precision_score,
    f1_score,
    matthews_corrcoef)

In [13]:
######## Modify here
exp_path = './experiments/'
exp_ids = ['exp_100k_gpu_nw16_0',
           'exp_100k_std_gpu_nw16_1']
comparison_id = 'Q1review'

exp_log = pd.read_excel(exp_path + '_experiments_log.xlsx', index_col='exp_id')
exp_log.head()

,exp_fullname,exp_path,start_time,end_time,input_data_path,protein_class,target_data,resolution,task,node_features,...,testing_f1,training_accuracy,validation_accuracy,testing_accuracy,training_precision,validation_precision,testing_precision,training_recall,validation_recall,testing_recall
exp_id,,,,,,,,,,,,,,,,,,,,,
exp_100k_pssm_rm_std_classw_bs64_net1_cl_allele_0,exp_100k_pssm_rm_std_classw_bs64_net1_cl_allel...,./experiments/exp_100k_pssm_rm_std_classw_bs64...,04/Apr/2023_09:51:54,04/Apr/2023_14:21:12,['/projects/0/einf2380/data/pMHCI/features_out...,I,BA,residue,classif,all,...,0.618,0.750,0.743,0.612,0.657,0.651,0.519,0.916,0.904,0.764
exp_100k_pssm_rm_std_classw_bs64_net1_0,exp_100k_pssm_rm_std_classw_bs64_net1_0_230404,./experiments/exp_100k_pssm_rm_std_classw_bs64...,04/Apr/2023_09:25:31,04/Apr/2023_13:49:34,['/projects/0/einf2380/data/pMHCI/features_out...,I,BA,residue,classif,all,...,0.726,0.797,0.782,0.770,0.812,0.785,0.766,0.702,0.694,0.689
exp_100k_pssm_rm_std_classw_bs64_net1_cl_peptide_0,exp_100k_pssm_rm_std_classw_bs64_net1_cl_pepti...,./experiments/exp_100k_pssm_rm_std_classw_bs64...,04/Apr/2023_09:47:12,04/Apr/2023_13:19:53,['/projects/0/einf2380/data/pMHCI/features_out...,I,BA,residue,classif,all,...,0.720,0.802,0.783,0.759,0.780,0.767,0.725,0.769,0.730,0.715
exp_100k_pssm_rm_std_classw_bs64_net1bn_cl_peptide_0,exp_100k_pssm_rm_std_classw_bs64_net1bn_cl_pep...,./experiments/exp_100k_pssm_rm_std_classw_bs64...,03/Apr/2023_15:17:35,03/Apr/2023_19:44:52,['/projects/0/einf2380/data/pMHCI/features_out...,I,BA,residue,classif,all,...,0.740,0.779,0.752,0.770,0.694,0.662,0.725,0.891,0.893,0.756
exp_100k_pssm_rm_std_classw_bs64_net1bn_0,exp_100k_pssm_rm_std_classw_bs64_net1bn_0_230403,./experiments/exp_100k_pssm_rm_std_classw_bs64...,03/Apr/2023_15:14:15,03/Apr/2023_19:10:14,['/projects/0/einf2380/data/pMHCI/features_out...,I,BA,residue,classif,all,...,0.742,0.793,0.780,0.771,0.757,0.781,0.738,0.781,0.697,0.745


In [14]:
######## Definitions used in the plotting
comparisons_path = os.path.join(exp_path, 'comparisons')
comparison_path = os.path.join(comparisons_path, comparison_id)

if not os.path.exists(comparisons_path):
    os.makedirs(comparisons_path)

if not os.path.exists(comparison_path):
    os.makedirs(comparison_path)
else:
    print(f'Folder comparisons/{comparison_id}/ already exists! \
          \nChange comparison_id if you want to save plots for a different comparison.')

def get_single_exp_df(exp_id, exp_log, exp_path):
    exp_fullname = exp_log.loc[exp_id].exp_fullname
    exp_path = os.path.join(exp_path, exp_fullname)
    output_path = os.path.join(exp_path, 'output')
    output_train = pd.read_hdf(os.path.join(output_path, 'output_exporter.hdf5'), key='training')
    output_test = pd.read_hdf(os.path.join(output_path, 'output_exporter.hdf5'), key='testing')
    df = pd.concat([output_train, output_test])
    df.sort_values(by=['epoch'], inplace = True)
    return df

Folder comparisons/Q1review/ already exists!           
Change comparison_id if you want to save plots for a different comparison.


In [15]:
######## Losses curves
palette = cycle(px.colors.qualitative.Plotly)
fig = go.Figure()

for exp_id in exp_ids:
    df = get_single_exp_df(exp_id, exp_log, exp_path)
    df_grouped = df.groupby(['phase', 'epoch']).mean().reset_index()[['phase', 'epoch', 'loss']]
    color = next(palette)
    df_train_plot = df_grouped[(df_grouped.phase =='training') & (df_grouped.epoch > 0)]
    df_valid_plot = df_grouped[(df_grouped.phase =='validation') & (df_grouped.epoch > 0)]

    fig.add_trace(go.Scatter(
        x=df_train_plot.epoch,
        y=df_train_plot.loss,
        mode = 'lines+markers',
        line=dict(color=color),
        name='training',
        legendgroup=exp_id,
        legendgrouptitle_text=exp_id))
    fig.add_trace(go.Scatter(
        x=df_valid_plot.epoch,
        y=df_valid_plot.loss,
        mode = 'lines+markers',
        line=dict(color=color, dash='dot'),
        name='validation',
        legendgroup=exp_id,
        legendgrouptitle_text=exp_id))
    
    epoch = exp_log.loc[exp_id].saved_epoch
    fig.add_trace(go.Scatter(
        x=[epoch],
        y=[exp_log.loc[exp_id].training_loss],
        mode = 'markers',
        marker_symbol = 'star',
        marker_color = 'firebrick',
        marker_size = 15,
        name = f'epoch {epoch}',
        legendgroup=exp_id,
        showlegend = False))

fig.update_layout(
    xaxis_title='Epoch #',
    yaxis_title='Loss',
    width=800, height=500,
    title='Loss vs epochs',
    title_x=0.5,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4),
    legend=dict(
        yanchor="top",
        xanchor="right",
        x=0.99
        )
)
fig.write_html(os.path.join(comparison_path, 'losses.html'))

/scratch-local/ccrocion/ipykernel_729883/3607644268.py:7: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/scratch-local/ccrocion/ipykernel_729883/3607644268.py:7: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [16]:
######## AUC and AUCPR curves
palette = cycle(px.colors.qualitative.Plotly)
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles = ['ROC Curves (AUC)', 'PR Curves (AUCPR)'],
    horizontal_spacing = 0.05)

for exp_id in exp_ids:
    df = get_single_exp_df(exp_id, exp_log, exp_path)
    df_plot = df[(df.epoch == 0) & (df.phase == 'testing')]
    y_true = df_plot.target
    y_score = np.array(df_plot.output.values.tolist())[:, 1]
    color = next(palette)

    df_plot_phase = df_plot
    y_true = df_plot_phase.target
    y_score = np.array(df_plot_phase.output.values.tolist())[:, 1]

    fpr_roc, tpr_roc, thr_roc = roc_curve(y_true, y_score)
    pr_pr, rec_pr, thr_pr = precision_recall_curve(y_true, y_score)

    name_roc = f'AUC={auc(fpr_roc, tpr_roc):.4f}'
    name_pr = f'AUCPR={average_precision_score(y_true, y_score):.4f}'

    fig.add_trace(go.Scatter(
        x=fpr_roc,
        y=tpr_roc,
        name=name_roc,
        line=dict(color=color),
        legendgroup=exp_id,
        legendgrouptitle_text=exp_id),
        row=1,
        col=1)
    fig.add_trace(go.Scatter(
        x=rec_pr,
        y=pr_pr,
        name=name_pr,
        line=dict(color=color),
        legendgroup=exp_id,
        legendgrouptitle_text=exp_id),
        row=1,
        col=2)

fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1,
    row = 1, col = 1
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=1, y1=0,
    row = 1, col = 2 
)
fig.update_layout(
    width=900, height=400,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    legend=dict(
        yanchor="top",
        y=1.05,
        xanchor="left",
        x=0.97
        ))
fig.update_xaxes(title_text="FPR", constrain='domain', scaleratio = 1, row=1, col=1)
fig.update_yaxes(title_text="TPR (Recall)", constrain='domain', scaleanchor = "x", scaleratio = 1, row=1, col=1)
fig.update_xaxes(title_text="Recall", constrain='domain', scaleanchor = "y", scaleratio = 1, row=1, col=2)
fig.update_yaxes(title_text="Precision", constrain='domain', scaleratio = 1, row=1, col=2)
fig.update_layout(
    title='AUC and AUCPR curves for testing set',
    title_x=0.5,
    margin={'t': 100})
fig.write_html(os.path.join(comparison_path, 'auc_aucpr_testing.html'))

In [17]:
######## F1 and MCC bar plots
palette = cycle(px.colors.qualitative.Plotly)
fig = go.Figure()

thr = 0.5

for exp_id in exp_ids:
    df = get_single_exp_df(exp_id, exp_log, exp_path)
    df_plot = df[(df.epoch == 0) & (df.phase == 'testing')]
    y_true = df_plot.target
    y_score = np.array(df_plot.output.values.tolist())[:, 1]
    y_pred = (y_score > thr)*1
    color = next(palette)
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    name_f1 = f'F1={f1:.4f}'
    name_mcc = f'MCC={mcc:.4f}'

    fig.add_trace(go.Bar(
        x=['F1', 'MCC'],
        y=[f1, mcc],
        name = name_f1 + ', ' + name_mcc,
        legendgroup=exp_id,
        legendgrouptitle_text=exp_id,
        marker_color = color
    ))

fig.update_yaxes(title_text="Score")
fig.update_layout(
    barmode='group',
    title='F1 and MCC scores for testing set',
    title_x=0.5,
    width=800, height=500)
fig.write_html(os.path.join(comparison_path, 'f1_mcc_thr.5_testing.html'))

In [18]:
######## Timings
fig = go.Figure()

for exp_id in exp_ids:
    start = exp_log.loc[exp_id].start_time
    end = exp_log.loc[exp_id].end_time
    start_dt = datetime.strptime(start, '%d/%b/%Y_%H:%M:%S')
    end_dt = datetime.strptime(end, '%d/%b/%Y_%H:%M:%S')
    time = end_dt - start_dt

    fig.add_trace(go.Bar(
        x=[exp_id],
        y=[time.seconds],
        name = str(time) + ' hours'
    ))

fig.update_yaxes(title_text="Seconds")
fig.update_layout(
    barmode='group',
    title='Total timings for the experiments',
    title_x=0.5,
    width=600, height=500)
fig.write_html(os.path.join(comparison_path, 'timings.html'))